**Plant Disease Detection**

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Older but sometimes useful
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#Configurations
dataset_dir = 'plants/plants'

# Choose which image type to train on for THIS model instance
# You will typically train a separate model for each type ('color', 'grayscale', 'segmented')
image_type_to_train = 'color' # <-- CHANGE THIS for 'grayscale' or 'segmented'

train_dir = os.path.join(dataset_dir, 'train', image_type_to_train)
test_dir = os.path.join(dataset_dir, 'test', image_type_to_train)

Model Parameter

In [4]:
# Model parameters
IMG_WIDTH = 224 # Standard size for ResNet50
IMG_HEIGHT = 224
BATCH_SIZE = 32
EPOCHS_PHASE1 = 10 # Fewer epochs for training only top layers
EPOCHS_PHASE2 = 20 # More epochs for fine-tuning
LEARNING_RATE_PHASE1 = 0.001
LEARNING_RATE_PHASE2 = 0.0001 # Much smaller learning rate for fine-tuning

GPU Setup

In [5]:
print("Checking for GPU availability...")
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Use MirroredStrategy for potential single or multi-GPU training
        strategy = tf.distribute.MirroredStrategy()
        print(f"Number of GPUs available: {len(gpus)}")
        print("Using distribution strategy: MirroredStrategy")  # Updated this line
        # You can configure GPU memory growth if needed
        # for gpu in gpus:
        #     tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"Error initializing GPU: {e}")
        print("Falling back to CPU.")
        strategy = tf.distribute.get_strategy()  # Default strategy (CPU)
else:
    print("No GPU devices found. Using CPU.")
    strategy = tf.distribute.get_strategy()  # Default strategy (CPU)

Checking for GPU availability...
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of GPUs available: 1
Using distribution strategy: MirroredStrategy


Data Loading and preprocessing

In [6]:
print(f"Loading data for '{image_type_to_train}'...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical', # Use 'categorical' for one-hot encoding, 'int' for sparse labels
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False # No need to shuffle test data
)

Loading data for 'color'...
Found 43429 files belonging to 38 classes.
Found 10876 files belonging to 38 classes.


Classes

In [7]:
# Get class names
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(f"Found {NUM_CLASSES} classes: {class_names}")

Found 38 classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', '

Data Augmentation


In [8]:
# --- Data Augmentation ---
# Using Keras Preprocessing Layers (recommended in TF 2.x)
data_augmentation = keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2),
  # Add more augmentation layers as needed
])

# Apply data augmentation only to the training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Cache and prefetch data for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)



Model Building

In [9]:
print("\n--- Building Model within Distribution Strategy Scope ---")
with strategy.scope():
    # Load the pre-trained ResNet50 model
    # Use weights='imagenet' for pre-trained weights
    # include_top=False removes the final classification layer
    base_model = tf.keras.applications.ResNet50(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), # ResNet50 expects 3 color channels
        include_top=False,
        weights='imagenet'
    )

    # Freeze the base model layers initially
    base_model.trainable = False

    # Create the model architecture
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    # Preprocessing: ResNet models expect input normalized in a specific way (ImageNet preprocessing)
    # Using the built-in preprocessing layer is convenient
    # This layer should be part of the model and will run on the GPU
    x = tf.keras.applications.resnet50.preprocess_input(inputs)

    # Pass the inputs through the base model
    # Set training=False when using base_model as a fixed feature extractor in Phase 1
    x = base_model(x, training=False)

    # Add the new classification layers
    x = GlobalAveragePooling2D()(x) # Reduces spatial dimensions
    x = Dropout(0.5)(x) # Add dropout for regularization
    outputs = Dense(NUM_CLASSES, activation='softmax')(x) # Final layer with 38 units and softmax activation

    model = Model(inputs, outputs)

    # --- Compile the Model (Phase 1) ---
    print("\n--- Compiling Model (Phase 1: Training Top Layers) ---")
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_PHASE1),
                  loss='categorical_crossentropy', # Matches label_mode='categorical'
                  metrics=['accuracy',
                            tf.keras.metrics.Precision(),
                            tf.keras.metrics.Recall()])

model.summary() # Print model architecture



--- Building Model within Distribution Strategy Scope ---
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

Training Model

In [10]:
# --- Training Phase 1 ---
print("\n--- Starting Training Phase 1 ---")

# Define callbacks
checkpoint_filepath_phase1 = f'best_model_{image_type_to_train}_phase1.h5'
model_checkpoint_callback_phase1 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_phase1,
    save_best_only=True,        # Save only the model with the best validation accuracy
    monitor='val_accuracy',      # Metric to monitor
    mode='max',                 # We want to maximize validation accuracy
    verbose=1
)

early_stopping_callback_phase1 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # Monitor validation loss
    patience=5,              # Stop if val_loss doesn't improve for 5 epochs
    restore_best_weights=True # Restore weights from the best epoch
)


history_phase1 = model.fit(
    train_ds,
    epochs=EPOCHS_PHASE1,
    validation_data=test_ds, # Use the test set for validation
    callbacks=[model_checkpoint_callback_phase1, early_stopping_callback_phase1]
)


--- Starting Training Phase 1 ---
Epoch 1/10
1358/1358 [==============================] - ETA: 0s - loss: 0.6141 - accuracy: 0.8236 - precision: 0.9169 - recall: 0.7507
Epoch 1: val_accuracy improved from -inf to 0.91596, saving model to best_model_color_phase1.h5
1358/1358 [==============================] - 323s 233ms/step - loss: 0.6141 - accuracy: 0.8236 - precision: 0.9169 - recall: 0.7507 - val_loss: 0.2582 - val_accuracy: 0.9160 - val_precision: 0.9386 - val_recall: 0.8949
Epoch 2/10
1358/1358 [==============================] - ETA: 0s - loss: 0.2703 - accuracy: 0.9158 - precision: 0.9395 - recall: 0.8934
Epoch 2: val_accuracy improved from 0.91596 to 0.93371, saving model to best_model_color_phase1.h5
1358/1358 [==============================] - 233s 171ms/step - loss: 0.2703 - accuracy: 0.9158 - precision: 0.9395 - recall: 0.8934 - val_loss: 0.2045 - val_accuracy: 0.9337 - val_precision: 0.9449 - val_recall: 0.9231
Epoch 3/10
1357/1358 [============================>.] - ETA: 0

Loading the Model

In [11]:
# Load the best model from Phase 1 (ensures we start fine-tuning from the best point)
print(f"\nLoading best model from Phase 1: {checkpoint_filepath_phase1}")
# Loading weights also happens within the strategy's context usually, but explicit load is fine
model.load_weights(checkpoint_filepath_phase1)



Loading best model from Phase 1: best_model_color_phase1.h5


Fine Tuning the Model Phase - 2

In [12]:
# --- Fine-tuning Setup (Phase 2) ---
print("\n--- Setting up Fine-tuning (Phase 2) ---")

# Unfreeze the base model (or parts of it)
base_model.trainable = True

# Decide how many layers to fine-tune. It's common to fine-tune the later layers
# Let's find the number of layers to unfreeze. ResNet50 has ~175 layers in the base model.
# You can choose to unfreeze the last block or last few blocks.
# Example: Unfreeze the last ~30 layers (adjust this number based on experimentation)
# You can print base_model.summary() to see layer names and count
fine_tune_from_layer = -30 # Fine-tune the last 30 layers

# Freeze all layers except the last `fine_tune_from_layer`
for layer in base_model.layers[:len(base_model.layers) + fine_tune_from_layer]:
    layer.trainable = False

# Recompile the model with a much lower learning rate
# Recompilation MUST happen inside the strategy scope if you modify trainable status
with strategy.scope():
    print("\n--- Recompiling Model (Phase 2: Fine-tuning) ---")
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_PHASE2), # Use a very low learning rate
                  loss='categorical_crossentropy',
                  metrics=['accuracy',
                            tf.keras.metrics.Precision(),
                            tf.keras.metrics.Recall()])

model.summary() # Print model architecture again to see which layers are trainable




--- Setting up Fine-tuning (Phase 2) ---

--- Recompiling Model (Phase 2: Fine-tuning) ---
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                   

Training the Model Phase 2

In [13]:
print("\n--- Starting Training Phase 2 (Fine-tuning) ---")

# Define callbacks for Phase 2
checkpoint_filepath_phase2 = f'best_model_{image_type_to_train}_phase2.h5'
model_checkpoint_callback_phase2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_phase2,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

early_stopping_callback_phase2 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # or 'val_accuracy'
    patience=10,         # More patience for fine-tuning
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1, # Reduce learning rate by a factor of 10
    patience=7, # If val_loss doesn't improve for 7 epochs
    min_lr=0.000001, # Minimum learning rate
    verbose=1
)


history_phase2 = model.fit(
    train_ds,
    epochs=EPOCHS_PHASE2,
    validation_data=test_ds,
    callbacks=[model_checkpoint_callback_phase2, early_stopping_callback_phase2, reduce_lr]
)



--- Starting Training Phase 2 (Fine-tuning) ---
Epoch 1/20
1358/1358 [==============================] - ETA: 0s - loss: 0.1847 - accuracy: 0.9402 - precision_1: 0.9520 - recall_1: 0.9308
Epoch 1: val_accuracy improved from -inf to 0.93288, saving model to best_model_color_phase2.h5
1358/1358 [==============================] - 241s 174ms/step - loss: 0.1847 - accuracy: 0.9402 - precision_1: 0.9520 - recall_1: 0.9308 - val_loss: 0.2188 - val_accuracy: 0.9329 - val_precision_1: 0.9370 - val_recall_1: 0.9294 - lr: 1.0000e-04
Epoch 2/20
1358/1358 [==============================] - ETA: 0s - loss: 0.0972 - accuracy: 0.9687 - precision_1: 0.9729 - recall_1: 0.9638
Epoch 2: val_accuracy improved from 0.93288 to 0.96331, saving model to best_model_color_phase2.h5
1358/1358 [==============================] - 235s 173ms/step - loss: 0.0972 - accuracy: 0.9687 - precision_1: 0.9729 - recall_1: 0.9638 - val_loss: 0.1115 - val_accuracy: 0.9633 - val_precision_1: 0.9656 - val_recall_1: 0.9615 - lr: 1

Phase 2 best model

In [14]:
# Load the best model from Phase 2 for final evaluation
print(f"\nLoading best model after Fine-tuning: {checkpoint_filepath_phase2}")
# Loading the final model for evaluation
best_model = tf.keras.models.load_model(checkpoint_filepath_phase2)


Loading best model after Fine-tuning: best_model_color_phase2.h5


Final Evaluation

In [15]:
print("\n--- Evaluating the Final Model on the Test Set ---")
# Evaluation also happens using the strategized model
loss, accuracy, precision, recall = best_model.evaluate(test_ds)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")


--- Evaluating the Final Model on the Test Set ---
340/340 [==============================] - 34s 98ms/step - loss: 0.0588 - accuracy: 0.9851 - precision_1: 0.9860 - recall_1: 0.9845
Test Loss: 0.0588
Test Accuracy: 0.9851
Test Precision: 0.9860
Test Recall: 0.9845


Detailed Metrics

In [17]:
# print("\n--- Generating Classification Report and Confusion Matrix ---")

# # Get true labels and predictions
# y_true = []
# y_pred_probs = []

# # Use predict on the dataset. This will also utilize the GPU via the strategy
# # Predict might require converting dataset to numpy or iterating
# # A common way to get predictions for metrics:
# test_images = np.concatenate([x for x, y in test_ds], axis=0)
# test_labels = np.concatenate([y for x, y in test_ds], axis=0)

# y_true = np.argmax(test_labels, axis=1)
# y_pred_probs = best_model.predict(test_images) # Prediction happens on GPU
# y_pred = np.argmax(y_pred_probs, axis=1)


# --- Final Evaluation ---
print("\n--- Evaluating the Final Model on the Test Set ---")
# Evaluation also happens using the strategized model
# Evaluate using the dataset directly
loss, accuracy, precision, recall = best_model.evaluate(test_ds)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# --- Detailed Metrics (Optional but Recommended) ---
print("\n--- Generating Classification Report and Confusion Matrix ---")

# Get true labels and predictions using the dataset directly
# Predict using the dataset. This will also utilize the GPU via the strategy.
# The predict method on a dataset returns predictions for each batch.
# We need to collect these predictions and the corresponding true labels.

y_pred_probs = best_model.predict(test_ds) # Prediction happens on GPU, returns batched predictions
y_pred = np.argmax(y_pred_probs, axis=1) # Convert probabilities to predicted class indices

# To get the true labels in the correct order, iterate through the dataset again
# Since test_ds has shuffle=False, iterating through it will give batches
# in the same order as predict() processed them.
y_true = []
for images, labels in test_ds:
    # labels are already one-hot encoded, convert back to integer indices
    y_true.extend(tf.argmax(labels, axis=1).numpy())

y_true = np.array(y_true) # Convert list to numpy array for scikit-learn

# Classification Report
print("\nClassification Report:")
# Ensure class_names are in the same order as the dataset indexed them
print(classification_report(y_true, y_pred, target_names=class_names))


--- Evaluating the Final Model on the Test Set ---
340/340 [==============================] - 35s 102ms/step - loss: 0.0588 - accuracy: 0.9851 - precision_1: 0.9860 - recall_1: 0.9845
Test Loss: 0.0588
Test Accuracy: 0.9851
Test Precision: 0.9860
Test Recall: 0.9845

--- Generating Classification Report and Confusion Matrix ---
340/340 [==============================] - 34s 94ms/step

Classification Report:
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.98      1.00      0.99       126
                                 Apple___Black_rot       0.98      1.00      0.99       125
                          Apple___Cedar_apple_rust       1.00      0.98      0.99        55
                                   Apple___healthy       0.99      1.00      1.00       329
                               Blueberry___healthy       0.99      1.00      1.00       301
          Cherry_(including_sour)__

Classification Report

In [18]:
# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion Matrix (can be large for 38 classes, but useful)
# print("\nConfusion Matrix:")
# print(confusion_matrix(y_true, y_pred))

# You can also plot the confusion matrix if needed


Classification Report:
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.98      1.00      0.99       126
                                 Apple___Black_rot       0.98      1.00      0.99       125
                          Apple___Cedar_apple_rust       1.00      0.98      0.99        55
                                   Apple___healthy       0.99      1.00      1.00       329
                               Blueberry___healthy       0.99      1.00      1.00       301
          Cherry_(including_sour)___Powdery_mildew       1.00      1.00      1.00       211
                 Cherry_(including_sour)___healthy       1.00      1.00      1.00       171
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.87      0.96      0.91       103
                       Corn_(maize)___Common_rust_       1.00      1.00      1.00       239
               Corn_(maize)___Northern_Leaf_Blight     

Plotting training history

In [19]:

def plot_history(history_phase1, history_phase2):
    acc1 = history_phase1.history['accuracy']
    val_acc1 = history_phase1.history['val_accuracy']
    loss1 = history_phase1.history['loss']
    val_loss1 = history_phase1.history['val_loss']

    acc2 = history_phase2.history['accuracy']
    val_acc2 = history_phase2.history['val_accuracy']
    loss2 = history_phase2.history['loss']
    val_loss2 = history_phase2.history['val_loss']

    # Combine histories
    total_acc = acc1 + acc2
    total_val_acc = val_acc1 + val_acc2
    total_loss = loss1 + loss2
    total_val_loss = val_loss1 + val_loss2

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.axvline(x=EPOCHS_PHASE1-1, color='r', linestyle='--', label='Phase 1 End')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])

    plt.subplot(1, 2, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.axvline(x=EPOCHS_PHASE1-1, color='r', linestyle='--', label='Phase 1 End')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    # Calculate y-axis limits based on actual data
    max_loss = max(max(total_loss), max(total_val_loss)) if total_loss and total_val_loss else 1
    plt.ylim([0, max_loss * 1.1])


    plt.show()

# Call the plotting function if matplotlib is available
# plot_history(history_phase1, history_phase2) # Uncomment to show plots

print(f"\nTraining for '{image_type_to_train}' completed. Best model saved to {checkpoint_filepath_phase2}")
print("Remember to repeat this process or adapt the code for 'grayscale' and 'segmented' data.")


Training for 'color' completed. Best model saved to best_model_color_phase2.h5
Remember to repeat this process or adapt the code for 'grayscale' and 'segmented' data.
